In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from torch.autograd import Variable


### Split label file to train, val and test

In [ ]:
train_labels = labels.loc[0:5999,:]
train_labels.to_csv('/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/labels.csv', sep=',')
val_labels = labels.loc[6000:7999,:]
val_labels.to_csv('/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/val/labels.csv', sep=',')
test_labels = labels.loc[8000:9999,:]
test_labels.to_csv('/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/test/labels.csv', sep=',')



### Customized dataloader for the dataset

In [21]:
class myDataset(Dataset):
    def __init__(self, csv_path, root_dir):
        self.root_dir = root_dir
        
        self.data_info = pd.read_csv(csv_path)
        #image path
        self.img_array = np.asarray(self.data_info.iloc[0:,1])
        #labels
        self.labels = np.asarray(self.data_info.iloc[0:,2], dtype=np.float32)
        #length of data
        self.data_len = len(self.data_info.index)
        #transformations on the batch
        self.transformations = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
    
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_array[idx] + '.jpeg')
        image = Image.open(img_path,'r')
        img_as_tensor = self.transformations(image)
        
        return {'image':img_as_tensor,'label':self.labels[idx]}
        #return sample
    

In [22]:
csv_path = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/labels.csv'
root_dir = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/'
trainset = myDataset(csv_path, root_dir)

In [20]:
labels = pd.read_csv(csv_path)
labels.head(5)

,Unnamed: 0,image,level
0,0,10_left,0
1,1,10_right,0
2,2,13_left,0
3,3,13_right,0
4,4,15_left,1


In [23]:
trainloader  = DataLoader(dataset=trainset,
                         batch_size=100,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
for i, batch in enumerate(trainloader):
    #print batch['label'].unsqueeze(1)
    #x = batch['image'].view(batch['image'].size(0), -1)
    print i, batch['label'].size()
    #break

0 torch.Size([100])
1 torch.Size([100])
2 torch.Size([100])
3 torch.Size([100])
4 torch.Size([100])
5 torch.Size([100])
6 torch.Size([100])
7 torch.Size([100])
8 torch.Size([100])
9 torch.Size([100])
10 torch.Size([100])
11 torch.Size([100])
12 torch.Size([100])
13 torch.Size([100])
14 torch.Size([100])
15 torch.Size([100])
16 torch.Size([100])
17 torch.Size([100])
18 torch.Size([100])
19 torch.Size([100])
20 torch.Size([100])
21 torch.Size([100])
22 torch.Size([100])
23 torch.Size([100])
24 torch.Size([100])
25 torch.Size([100])
26 torch.Size([100])
27 torch.Size([100])
28 torch.Size([100])
29 torch.Size([100])
30 torch.Size([100])
31 torch.Size([100])
32 torch.Size([100])
33 torch.Size([100])
34 torch.Size([100])
35 torch.Size([100])
36 torch.Size([100])
37 torch.Size([100])
38 torch.Size([100])
39 torch.Size([100])
40 torch.Size([100])
41 torch.Size([100])
42 torch.Size([100])
43 torch.Size([100])
44 torch.Size([100])
45 torch.Size([100])
46 torch.Size([100])
47 torch.Size([100])
48

### Plot a batch to make sure it's loaded properly

In [ ]:
trainloader  = DataLoader(dataset=trainset,
                         batch_size=5,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)

def plot_batch(sample_batch):
    images_batch = sample_batch['image']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')
    
for i, sample_batch in enumerate(trainloader):
    if i ==3:
        plot_batch(sample_batch)
        plt.show()
        break

### Model architecture

In [15]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv7 = nn.Conv2d(128, 256, 3)
        self.conv8 = nn.Conv2d(256, 256, 3)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256*12*12,4096)
        self.fc2 = nn.Linear(4096, 5)
    
    def forward(self, input):
        x = self.pool1(self.conv2(F.relu(self.conv1(input))))
        x = self.pool2(self.conv4(F.relu(self.conv3(x))))
        x = self.pool3(self.conv6(F.relu(self.conv5(x))))
        x = self.pool4(self.conv8(F.relu(self.conv7(x))))
        #flatten the tensor for the FC
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
    

### Training

In [16]:
#torch.cuda.empty_cache()
use_cuda = torch.cuda.is_available()
learning_rate = 0.0001
l2_penalty = 0.001
batch_size = 5
num_epochs = 60
display_step = 10

trainset = myDataset(csv_path, root_dir)
trainloader  = DataLoader(dataset=trainset,
                         batch_size=5,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
model = ConvNet().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
#scheduler_model = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.0)
criterion = nn.NLLLoss().cuda()
losses = []
for epoch in range(num_epochs):
    loss_ = 0.
    predicted = []
    ground_truth = []
    for batch_idx, batch in enumerate(trainloader):
        data = batch['image']
        data = Variable(data)
        labels = batch['label']
        labels = labels.long()
        labels = Variable(labels)
        if use_cuda:
            data = data.cuda()
            labels = labels.cuda()
            #print 'gpu is working'
        #if labels.data.size()[0] == batch_size:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            print 'loss deledted'
            loss_ += loss.data[0]
            loss.backward()
            #del output
            
            optimizer.step()
            #torch.cuda.empty_cache()
        if batch_idx % display_step == 0:
            print 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item())
            x = np.array(range(num_epochs))
            plt.plot(x, losses)
            plt.xlabel('Number of Epochs')
            plt.ylabel('loss')
            plt.title('Training Loss')
            plt.show()
            
        print 'batch number: {} ===== loss in this iteration: {}'.format(batch_idx, loss.item())


loss deledted


/home/eslam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524577177097/work/aten/src/THC/generic/THCTensorMath.cu:35

### Testing 

In [ ]:
test_loss = 0
correct = 0
test_data_csv = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/test/labels.csv'
root_dir_test = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/test/'
testset = myDataset(test_data_csv, root_dir_test)
testloader  = DataLoader(dataset=testset,
                         batch_size=2,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
with torch.no_grad():
    for batch_idx, batch in enumerate(testloader):
        data, target = batch['image'], batch['label']
        target = target.long()
        data, target = data.cuda(), target.cuda()
        output = model(data)
        '''test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        '''
        loss = criterion(output, target)
        _,pred = torch.max(output.data, 1)
        print pred
        test_loss = loss.data[0]
        correct += (pred == target.data).sum()
        print '=====> Number of batch <====={}'.format(batch_idx)
        print'\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader.dataset),
            100. * correct / len(testloader.dataset))
#test_loss /= len(testloader.dataset)
        #break


### plot training loss 

In [ ]:
x = np.array(range(num_epochs))
plt.plot(x, losses)
plt.xlabel('Number of Epochs')
plt.ylabel('loss')
plt.title('Training Loss')
plt.show()

In [7]:
def pretty_size(size):
    """Pretty prints a torch.Size object"""
    assert(isinstance(size, torch.Size))
    return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
    """Prints a list of the Tensors being tracked by the garbage collector."""
    import gc
    total_size = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if not gpu_only or obj.is_cuda:
                    print("%s:%s%s %s" % (type(obj).__name__, 
                                          " GPU" if obj.is_cuda else "",
                                          " pinned" if obj.is_pinned else "",
                                          pretty_size(obj.size())))
                    total_size += obj.numel()
            elif hasattr(obj, "data") and torch.is_tensor(obj.data):
                if not gpu_only or obj.is_cuda:
                    print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
                                                   type(obj.data).__name__, 
                                                   " GPU" if obj.is_cuda else "",
                                                   " pinned" if obj.data.is_pinned else "",
                                                   " grad" if obj.requires_grad else "", 
                                                   " volatile" if obj.volatile else "",
                                                   pretty_size(obj.data.size())))
                    total_size += obj.data.numel()
        except Exception as e:
            pass        
    print("Total size:", total_size)

In [13]:
dump_tensors()

Tensor: GPU pinned 
Tensor: GPU pinned 
Tensor: GPU pinned 
('Total size:', 3)


In [11]:
del optimizer
torch.cuda.empty_cache()